In [ ]:
# ICD codes for AMI:
# ICD-9: 410.*
# ICD-10: I21.*, I22.*

# Define the cohort, get all id with patient that match the icd code
CREATE OR REPLACE TABLE `comp90089_group_assignment.ami_icd` AS
SELECT DISTINCT icu.subject_id, icu.hadm_id, icu.stay_id, diag.icd_code
FROM `physionet-data.mimiciv_3_1_icu.icustays` icu
JOIN `physionet-data.mimiciv_3_1_hosp.diagnoses_icd` diag
ON icu.hadm_id = diag.hadm_id
WHERE (diag.icd_code LIKE '410%' OR diag.icd_code LIKE 'I21%' OR diag.icd_code LIKE 'I22%');

# Find patient with elevated troponin levels within the first 24 hours of ICU admission.
CREATE OR REPLACE TABLE `comp90089_group_assignment.troponin_24h` AS
SELECT DISTINCT l.subject_id, l.hadm_id, l.charttime, icu.intime
  FROM `physionet-data.mimiciv_3_1_hosp.labevents` l
  JOIN `physionet-data.mimiciv_3_1_icu.icustays` icu
  ON l.subject_id = icu.subject_id
WHERE l.itemid IN (
    # Get itemid related to troponin
    SELECT itemid
      FROM `physionet-data.mimiciv_3_1_hosp.d_labitems`
      WHERE LOWER(label) LIKE '%troponin%'
)
  AND l.charttime BETWEEN icu.intime AND icu.intime + INTERVAL 24 HOUR
  AND l.valuenum IS NOT NULL;

CREATE OR REPLACE TABLE `comp90089_group_assignment.ami_verified` AS
SELECT DISTINCT
  icd.subject_id,
  icd.hadm_id,
  icd.stay_id
FROM `comp90089_group_assignment.ami_icd` icd
JOIN `comp90089_group_assignment.troponin_24h` trop
  ON icd.subject_id = trop.subject_id
  AND icd.hadm_id = trop.hadm_id;

# Demographics
CREATE OR REPLACE TABLE `comp90089_group_assignment.demo_features` AS
SELECT p.subject_id,a.hadm_id,
  EXTRACT(YEAR FROM a.admittime) - p.anchor_year AS age,
  p.gender,
  a.race
FROM `physionet-data.mimiciv_3_1_hosp.patients` p
JOIN `physionet-data.mimiciv_3_1_hosp.admissions` a
ON p.subject_id = a.subject_id;

# Comorbidities - Hypertension (401–405), Diabetes (250), COPD (491–496), CKD (585), Heart Failure (428)
CREATE OR REPLACE TABLE `comp90089_group_assignment.comorbid_features` AS
SELECT
  hadm_id,
  MAX(CASE WHEN icd_code LIKE '401%' OR icd_code LIKE 'I10%' THEN 1 ELSE 0 END) AS hypertension,
  MAX(CASE WHEN icd_code LIKE '250%' OR icd_code LIKE 'E11%' THEN 1 ELSE 0 END) AS diabetes,
  MAX(CASE WHEN icd_code LIKE '491%' OR icd_code LIKE 'J44%' THEN 1 ELSE 0 END) AS copd,
  MAX(CASE WHEN icd_code LIKE '585%' OR icd_code LIKE 'N18%' THEN 1 ELSE 0 END) AS ckd,
  MAX(CASE WHEN icd_code LIKE '428%' OR icd_code LIKE 'I50%' THEN 1 ELSE 0 END) AS heart_failure
FROM `physionet-data.mimiciv_3_1_hosp.diagnoses_icd`
GROUP BY hadm_id;

# Vital Signs
CREATE OR REPLACE TABLE `comp90089_group_assignment.vital_hr` AS
SELECT
  ce.subject_id,
  ce.stay_id,
  AVG(ce.valuenum) AS hr_mean,
  MIN(ce.valuenum) AS hr_min,
  MAX(ce.valuenum) AS hr_max
FROM `physionet-data.mimiciv_3_1_icu.chartevents` ce
JOIN `physionet-data.mimiciv_3_1_icu.icustays` icu
  ON ce.stay_id = icu.stay_id
JOIN `comp90089_group_assignment.ami_verified` ami
  ON ce.stay_id = ami.stay_id
WHERE ce.itemid IN (220045, 211)  -- Heart Rate
  AND ce.charttime BETWEEN icu.intime AND icu.intime + INTERVAL 24 HOUR
  AND ce.valuenum IS NOT NULL
GROUP BY ce.subject_id, ce.stay_id;

# Laboratory Results
CREATE OR REPLACE TABLE `comp90089_group_assignment.lab_features` AS
SELECT
  l.subject_id,
  l.hadm_id,
  AVG(CASE WHEN l.itemid = 50912 THEN l.valuenum END) AS creatinine,
  AVG(CASE WHEN l.itemid = 50813 THEN l.valuenum END) AS lactate,
  AVG(CASE WHEN l.itemid = 50931 THEN l.valuenum END) AS glucose,
  AVG(CASE WHEN l.itemid = 50983 THEN l.valuenum END) AS sodium,
  AVG(CASE WHEN l.itemid = 50971 THEN l.valuenum END) AS potassium
FROM `physionet-data.mimiciv_3_1_hosp.labevents` l
JOIN `comp90089_group_assignment.ami_verified` ami
  ON l.subject_id = ami.subject_id
JOIN `physionet-data.mimiciv_3_1_icu.icustays` icu
  ON ami.stay_id = icu.stay_id
WHERE l.charttime BETWEEN icu.intime AND icu.intime + INTERVAL 24 HOUR
GROUP BY l.subject_id, l.hadm_id;

# Early Interventions
CREATE OR REPLACE TABLE `comp90089_group_assignment.intervention_features` AS
SELECT
  ie.subject_id,
  ie.stay_id,
  MAX(
    CASE
      WHEN LOWER(di.label) LIKE '%norepinephrine%'
        OR LOWER(di.label) LIKE '%epinephrine%'
        OR LOWER(di.label) LIKE '%dopamine%'
        OR LOWER(di.label) LIKE '%phenylephrine%'
      THEN 1 ELSE 0
    END
  ) AS vasopressor_use
FROM `physionet-data.mimiciv_3_1_icu.inputevents` ie
JOIN `physionet-data.mimiciv_3_1_icu.d_items` di
  ON ie.itemid = di.itemid
GROUP BY ie.subject_id, ie.stay_id;

# died in icu
CREATE OR REPLACE TABLE `comp90089_group_assignment.icu_mortality` AS
SELECT
  icu.stay_id,
  CASE
    WHEN adm.hospital_expire_flag = 1
         AND p.dod IS NOT NULL
         AND p.dod BETWEEN DATE(icu.intime) AND DATE(icu.outtime)
    THEN 1 ELSE 0
  END AS icu_expire_flag
FROM `physionet-data.mimiciv_3_1_icu.icustays` icu
JOIN `physionet-data.mimiciv_3_1_hosp.admissions` adm
  ON icu.hadm_id = adm.hadm_id
JOIN `physionet-data.mimiciv_3_1_hosp.patients` p
  ON icu.subject_id = p.subject_id;

# when did they die
CREATE OR REPLACE TABLE `comp90089_group_assignment.post_discharge_mortality` AS
SELECT
  adm.subject_id,
  adm.hadm_id,
  CASE
    WHEN p.dod IS NOT NULL AND DATE_DIFF(p.dod, adm.dischtime, DAY) BETWEEN 1 AND 30 THEN 1
    ELSE 0
  END AS death_30d_post_discharge
FROM `physionet-data.mimiciv_3_1_hosp.admissions` adm
LEFT JOIN `physionet-data.mimiciv_3_1_hosp.patients` p
  ON adm.subject_id = p.subject_id
WHERE adm.hadm_id IN (SELECT hadm_id FROM `comp90089_group_assignment.ami_verified`);


# combine
CREATE OR REPLACE TABLE `comp90089_group_assignment.ami_features_24h` AS
SELECT
  a.subject_id, a.hadm_id, a.stay_id, adm.hospital_expire_flag,
  d.age, d.gender, d.race,
  c.hypertension, c.diabetes, c.copd, c.ckd, c.heart_failure,
  v.hr_mean, v.hr_min, v.hr_max,
  l.creatinine, l.lactate, l.glucose, l.sodium, l.potassium,
  i.vasopressor_use, icu.icu_expire_flag, post.death_30d_post_discharge,
FROM `comp90089_group_assignment.ami_verified` a
LEFT JOIN `physionet-data.mimiciv_3_1_hosp.admissions` adm
  ON a.hadm_id = adm.hadm_id
LEFT JOIN `comp90089_group_assignment.demo_features` d
  ON a.subject_id = d.subject_id
  AND a.hadm_id = d.hadm_id
LEFT JOIN `comp90089_group_assignment.comorbid_features` c
  ON a.hadm_id = c.hadm_id
LEFT JOIN `comp90089_group_assignment.vital_hr` v
  ON a.stay_id = v.stay_id
LEFT JOIN `comp90089_group_assignment.lab_features` l
  ON a.hadm_id = l.hadm_id
LEFT JOIN `comp90089_group_assignment.intervention_features` i
  ON a.stay_id = i.stay_id
LEFT JOIN `comp90089_group_assignment.icu_mortality` icu
  ON a.stay_id = icu.stay_id
LEFT JOIN `comp90089_group_assignment.post_discharge_mortality` post
  ON a.hadm_id = post.hadm_id;